# Simulate NIRCam Images

## Using MIRAGE and the JWST Pipeline

https://github.com/spacetelescope/mirage/blob/master/examples/Imaging_simulator_use_examples.ipynb

Here we use MIRAGE to simulate NIRCam imaging based on HST observations of the galaxy cluster MACS0647+70.  
Our full APT program executes 160 exposures = 4 dithers x 4 filters x 10 detectors.
Here we simulate images in one filter in one NIRCam Module A.
For the short wavelength filters, that will be 4 dithers x 4 detectors = 16 images.
For the long wavelength filters,  that will be 4 dithers x 1 detector  =  4 images.

Inputs:  
APT file outputs: .xml, .pointing
Galaxy catalog incl. RA, Dec, Sersic fit parameters, magnitude

Outputs:  
MIRAGE Simulated NIRCam FITS images (raw.fits; don't save linear.fits)

Next up:
JWST Pipeline -> Reduced NIRCam FITS images and catalog (id2.fits, cat.ecsv)

# Outstanding issues / room for improvement:

- Darks
- Stars too small

# Import general

In [1]:
import os
home = os.path.expanduser("~")
from glob import glob
import numpy as np
from astropy.io import fits

# Inputs

In [2]:
filt_to_process = 'F277W'
# Uncomment CEERS tweaks below next time you run this
obs_to_process = ['010', '020']  # 2 NIRCam epochs
module_to_process = 'A'
exposures_to_process = 'all'  # [1]  # e.g., 1, 2, 3, 4 -OR- 'all' to process all

In [3]:
# Specify the xml and pointing files exported from APT
APT_input_dir     = './inputs/'  # APT
APT_file = os.path.join(APT_input_dir, 'macs0647_NIRCam')
xml_file      = APT_file + '.xml'
pointing_file = APT_file + '.pointing'

In [4]:
# Source catalogs
target = 'MACS0647+7015'  # must correspond to observed target in APT file!!
# Otherwise may throw error when calculating catalog seed image
cat_dict = {target:{}}
galaxy_catalog_file = 'MACS0647_MIRAGE_galaxy_catalog_%s.cat' % filt_to_process
galaxy_catalog_file = os.path.join(APT_input_dir, galaxy_catalog_file)
cat_dict[target]['galaxy'] = galaxy_catalog_file
#cat_dict[target]['point_source'] = 'imaging_example_data/ptsrc_catalog.cat'

In [5]:
reffile_defaults = 'crds'  # Reference file values: crds or crds_full_name
cosmic_rays = {'library': 'SUNMAX', 'scale': 1.0}  # Cosmic ray library and rate
background = 'medium'
pav3 = 12.5  # telescope roll angle
dates = '2022-10-31'  # won't be used by MIRAGE, but will be added to FITS headers

# Outputs

In [12]:
#output_dir     = './yaml/'    # yaml files
image_dir = './images/'  # simulated images
image_dir = os.path.join(image_dir, filt_to_process)
output_dir = simulation_dir = image_dir
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

#yaml_dir = image_dir
yaml_dir  = './yaml/'    # yaml files
yaml_dir = os.path.join(yaml_dir, filt_to_process)
if not os.path.exists(yaml_dir):
    os.makedirs(yaml_dir)

#datatype = 'linear, raw'  # Save both raw (for JWST pipeline) and linear (processed except for dark current subtraction)
datatype = 'raw'  # Save raw images only for JWST pipeline

# Import MIRAGE

In [7]:
import mirage
os.environ["CRDS_DATA"] = "$HOME/crds_cache"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"
os.environ["MIRAGE_DATA"] = "/ifs/jwst/wit/mirage_data"  # internal to STScI
mirage.__version__

'2.2.1'

In [8]:
if 0:  # for users external to STScI
    # Download 343 GB of files (will take some time!)
    from mirage.reference_files import downloader
    download_path = os.path.join(home, 'MIRAGE', 'data')
    os.makedirs(download_path, exist_ok=True)
    downloader.download_reffiles(download_path, instrument='nircam', dark_type='both',  # linearized
                                 skip_darks=False, skip_cosmic_rays=False, skip_psfs=False, skip_grism=True)
    
    os.environ["MIRAGE_DATA"] = download_path

In [9]:
# mirage imports
from mirage import imaging_simulator
from mirage.seed_image import catalog_seed_image
from mirage.dark import dark_prep
from mirage.ramp_generator import obs_generator
from mirage.yaml import yaml_generator

**WARNING**: LOCAL JWST PRD VERSION PRDOPSSOC-044 DOESN'T MATCH THE CURRENT ONLINE VERSION PRDOPSSOC-045
Please consider updating pysiaf, e.g. pip install --upgrade pysiaf or conda update pysiaf
2022-04-21 23:13:56,446 - numexpr.utils - INFO - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-04-21 23:13:56,448 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [10]:
import yaml

# Import JWST Pipeline

In [11]:
import jwst
from jwst.pipeline import Detector1Pipeline, Image2Pipeline
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from jwst.associations import asn_from_list
from jwst.pipeline import calwebb_image3
jwst.__version__

'1.4.6'

# Run

In [13]:
# Run the yaml generator (takes a minute or so)
yam = yaml_generator.SimInput(input_xml=xml_file, pointing_file=pointing_file, catalogs=cat_dict, 
                              cosmic_rays=cosmic_rays, background=background, roll_angle=pav3, dates=dates, 
                              reffile_defaults=reffile_defaults, datatype=datatype, verbose=True,
                              output_dir=yaml_dir, simdata_output_dir=image_dir)

# https://mirage-data-simulator.readthedocs.io/en/latest/dark_preparation.html#calibration-and-linearization
#yam.use_linearized_darks = False  # MIRAGE gave me an error
yam.use_linearized_darks = True

yam.create_inputs()

2022-04-21 23:18:49,314 - mirage.yaml.yaml_generator - INFO - Running yaml_generator....


2022-04-21 23:18:49,314 - stpipe - INFO - Running yaml_generator....


2022-04-21 23:18:49,314 - stpipe - INFO - Running yaml_generator....
2022-04-21 23:18:49,322 - mirage.yaml.yaml_generator - INFO - 


2022-04-21 23:18:49,322 - stpipe - INFO - 


2022-04-21 23:18:49,322 - stpipe - INFO - 
2022-04-21 23:18:49,325 - mirage.yaml.yaml_generator - INFO - using APT xml file: ./inputs/macs0647_NIRCam.xml


...

2022-04-21 23:20:58,261 - stpipe - INFO -     40 file(s)
2022-04-21 23:20:58,262 - mirage.yaml.yaml_generator - INFO - 160 output files written to: /Users/dcoe/NIRCam/MIRAGE/2022April/yaml/F277W


2022-04-21 23:20:58,262 - stpipe - INFO - 160 output files written to: /Users/dcoe/NIRCam/MIRAGE/2022April/yaml/F277W


2022-04-21 23:20:58,262 - stpipe - INFO - 160 output files written to: /Users/dcoe/NIRCam/MIRAGE/2022April/yaml/F277W
2022-04-21 23:20:58,263 - mirage.yaml.yaml_generator - INFO - Yaml generator complete


2022-04-21 23:20:58,263 - stpipe - INFO - Yaml generator complete


2022-04-21 23:20:58,263 - stpipe - INFO - Yaml generator complete


In [14]:
yam.use_linearized_darks

True

In [15]:
# 4 filters x 4 dithers x 10 detectors (as specified in APT)
yfiles = glob(os.path.join(yaml_dir,'jw*.yaml'))
yfiles = np.sort(yfiles)
len(yfiles)

160

In [16]:
# Print info about these files: filter and detector
for yamlfile in yfiles:
    with open(yamlfile, 'r') as infile:
        params = yaml.safe_load(infile)
    filt = params['Readout']['filter']
    detector = params['Readout']['array_name'][3:5]
    print(filt, detector, params['Output']['observation_number'], yamlfile)

F115W A1 010 ./yaml/F277W/jw01433010001_01101_00001_nrca1.yaml
F115W A2 010 ./yaml/F277W/jw01433010001_01101_00001_nrca2.yaml
F115W A3 010 ./yaml/F277W/jw01433010001_01101_00001_nrca3.yaml
F115W A4 010 ./yaml/F277W/jw01433010001_01101_00001_nrca4.yaml
F444W A5 010 ./yaml/F277W/jw01433010001_01101_00001_nrca5.yaml
F115W B1 010 ./yaml/F277W/jw01433010001_01101_00001_nrcb1.yaml
F115W B2 010 ./yaml/F277W/jw01433010001_01101_00001_nrcb2.yaml
F115W B3 010 ./yaml/F277W/jw01433010001_01101_00001_nrcb3.yaml
F115W B4 010 ./yaml/F277W/jw01433010001_01101_00001_nrcb4.yaml
F444W B5 010 ./yaml/F277W/jw01433010001_01101_00001_nrcb5.yaml
F115W A1 010 ./yaml/F277W/jw01433010001_01101_00002_nrca1.yaml
F115W A2 010 ./yaml/F277W/jw01433010001_01101_00002_nrca2.yaml
F115W A3 010 ./yaml/F277W/jw01433010001_01101_00002_nrca3.yaml
F115W A4 010 ./yaml/F277W/jw01433010001_01101_00002_nrca4.yaml
F444W A5 010 ./yaml/F277W/jw01433010001_01101_00002_nrca5.yaml
F115W B1 010 ./yaml/F277W/jw01433010001_01101_00002_nrc

F200W A2 020 ./yaml/F277W/jw01433020001_01101_00002_nrca2.yaml
F200W A3 020 ./yaml/F277W/jw01433020001_01101_00002_nrca3.yaml
F200W A4 020 ./yaml/F277W/jw01433020001_01101_00002_nrca4.yaml
F480M A5 020 ./yaml/F277W/jw01433020001_01101_00002_nrca5.yaml
F200W B1 020 ./yaml/F277W/jw01433020001_01101_00002_nrcb1.yaml
F200W B2 020 ./yaml/F277W/jw01433020001_01101_00002_nrcb2.yaml
F200W B3 020 ./yaml/F277W/jw01433020001_01101_00002_nrcb3.yaml
F200W B4 020 ./yaml/F277W/jw01433020001_01101_00002_nrcb4.yaml
F480M B5 020 ./yaml/F277W/jw01433020001_01101_00002_nrcb5.yaml
F200W A1 020 ./yaml/F277W/jw01433020001_01101_00003_nrca1.yaml
F200W A2 020 ./yaml/F277W/jw01433020001_01101_00003_nrca2.yaml
F200W A3 020 ./yaml/F277W/jw01433020001_01101_00003_nrca3.yaml
F200W A4 020 ./yaml/F277W/jw01433020001_01101_00003_nrca4.yaml
F480M A5 020 ./yaml/F277W/jw01433020001_01101_00003_nrca5.yaml
F200W B1 020 ./yaml/F277W/jw01433020001_01101_00003_nrcb1.yaml
F200W B2 020 ./yaml/F277W/jw01433020001_01101_00003_nrc

In [17]:
#params

In [18]:
# Print info about these files: filter and detector
for yamlfile in yfiles:
    with open(yamlfile, 'r') as infile:
        params = yaml.safe_load(infile)
    filt = params['Readout']['filter']
    detector = params['Readout']['array_name'][3:5]
    #if detector == 'A2':  # bad dark
    #    break
        
params

{'Inst': {'instrument': 'NIRCAM',
  'mode': 'imaging',
  'use_JWST_pipeline': True},
 'Readout': {'readpatt': 'SHALLOW4',
  'ngroup': 10,
  'nint': 1,
  'namp': 4,
  'resets_bet_ints': 1,
  'array_name': 'NRCB5_FULL',
  'intermediate_aperture': 'None',
  'PPS_aperture': 'NRCALL_FULL',
  'filter': 'F480M',
  'pupil': 'CLEAR'},
 'Reffiles': {'dark': 'None',
  'linearized_darkfile': '/ifs/jwst/wit/mirage_data/nircam/darks/linearized/B5/Linearized_Dark_and_SBRefpix_NRCNRCBLONG-DARK-60011525031_1_490_SE_2016-01-01T17h24m42_uncal.fits',
  'badpixmask': 'crds',
  'superbias': 'crds',
  'linearity': 'crds',
  'saturation': 'crds',
  'gain': 'crds',
  'pixelflat': 'crds',
  'illumflat': 'None',
  'astrometric': 'crds',
  'photom': 'crds',
  'ipc': 'crds',
  'invertIPC': True,
  'occult': 'None',
  'pixelAreaMap': 'crds',
  'transmission': 'crds',
  'subarray_defs': '/Users/dcoe/miniconda3/envs/python3.10/lib/python3.10/site-packages/mirage/config/NIRCam_subarray_definitions.list',
  'readpattde

In [19]:
int(params['Output']['exposure_number'])

4

In [20]:
if   module_to_process == 'A':
    yfiles = [yfile for yfile in yfiles if '_nrca' in yfile]  # Select only images in NIRCam Module A
elif module_to_process == 'B':
    yfiles = [yfile for yfile in yfiles if '_nrcb' in yfile]  # Select only images in NIRCam Module B

#a5files = [yfile for yfile in yfiles if 'a5.' in yfile] # Select only Module A Long Wavelength images: detector A5

In [21]:
#exposures_to_process = [1]

In [22]:
# Select filter observations
yamls_to_process = []
for yamlfile in yfiles:
    with open(yamlfile, 'r') as infile:
        params = yaml.safe_load(infile)
    yaml_filt = params['Readout']['filter']
    obs_num = params['Output']['observation_number']
    if obs_num in obs_to_process:
        if yaml_filt == filt_to_process:
            if (exposures_to_process == 'all') or (int(params['Output']['exposure_number']) in exposures_to_process):
                yamls_to_process.append(yamlfile)
        
len(yamls_to_process)

4

In [23]:
yamls_to_process

['./yaml/F277W/jw01433010001_01101_00009_nrca5.yaml',
 './yaml/F277W/jw01433010001_01101_00010_nrca5.yaml',
 './yaml/F277W/jw01433010001_01101_00011_nrca5.yaml',
 './yaml/F277W/jw01433010001_01101_00012_nrca5.yaml']

In [24]:
# Only create images that haven't been created already
yamls = np.sort(yamls_to_process)
yamls_to_process = []
for yamlfile in yamls:
    outfits = yamlfile.replace('.yaml', '_uncal.fits')
    already_did_it = os.path.exists(outfits)
    havent_done_it_yet = not already_did_it
    if havent_done_it_yet:
        yamls_to_process.append(yamlfile)

In [25]:
len(yamls_to_process) 

4

In [26]:
yamls_to_process = np.sort(yamls_to_process)
yamls_to_process

array(['./yaml/F277W/jw01433010001_01101_00009_nrca5.yaml',
       './yaml/F277W/jw01433010001_01101_00010_nrca5.yaml',
       './yaml/F277W/jw01433010001_01101_00011_nrca5.yaml',
       './yaml/F277W/jw01433010001_01101_00012_nrca5.yaml'], dtype='<U49')

# Create the simulated images (will take a while)

In [28]:
cal_images = []
for yfile in yamls_to_process:
    print(yfile)
    uncal_image = yfile.replace('.yaml', '_uncal.fits').replace(yaml_dir, image_dir)

    # MIRAGE
    # 3 (7) minutes per image at work (home on VPN)
    if not os.path.exists(uncal_image):
        m = imaging_simulator.ImgSim()
        m.paramfile = yfile
        m.create()
        
    hdu = fits.open(uncal_image)
    try:
        noutputs = hdu[0].header['NOUTPUTS']
    except:
        hdu[0].header['NOUTPUTS'] = 4
        hdu.writeto(uncal_image, overwrite=True)

./yaml/F277W/jw01433010001_01101_00009_nrca5.yaml
2022-04-21 23:23:18,903 - mirage.imaging_simulator - INFO - 


2022-04-21 23:23:18,903 - stpipe - INFO - 


2022-04-21 23:23:18,903 - stpipe - INFO - 
2022-04-21 23:23:18,910 - mirage.imaging_simulator - INFO - 


2022-04-21 23:23:18,910 - stpipe - INFO - 


2022-04-21 23:23:18,910 - stpipe - INFO - 
2022-04-21 23:23:18,916 - mirage.imaging_simulator - INFO - Running imaging_simulator....


...

2022-04-22 00:36:00,391 - stpipe - INFO - Observation generation complete.
2022-04-22 00:36:00,439 - mirage.imaging_simulator - INFO - 


2022-04-22 00:36:00,439 - stpipe - INFO - 


2022-04-22 00:36:00,439 - stpipe - INFO - 
2022-04-22 00:36:00,441 - mirage.imaging_simulator - INFO - Imaging simulator complete


2022-04-22 00:36:00,441 - stpipe - INFO - Imaging simulator complete


2022-04-22 00:36:00,441 - stpipe - INFO - Imaging simulator complete


The raw data file is now ready to be run through the [JWST calibration pipeline](https://jwst-pipeline.readthedocs.io/en/stable/) from the beginning. If dark current subtraction is not important for you, you can use Mirage's linear output, skip some of the initial steps of the pipeline, and begin by running the [Jump detection](https://jwst-pipeline.readthedocs.io/en/stable/jwst/jump/index.html?highlight=jump) and [ramp fitting](https://jwst-pipeline.readthedocs.io/en/stable/jwst/ramp_fitting/index.html) steps.

---
<a id='mult_sims'></a>
## Simulating Multiple Exposures

Each yaml file will simulate an exposure for a single pointing using a single detector. To simulate multiple exposures, or a single exposure with multiple detectors, multiple calls to the *imaging_simulator* must be made.

### In Series
```python
paramlist = [yaml_a1,yaml_a2,yaml_a3,yaml_a4,yaml_a5]

def many_sim(paramlist):
    '''Function to run many simulations in series
    '''
    for file in paramlist:
        m = imaging_simulator.ImgSim()
        m.paramfile = file
        m.create()
```

### In Parallel

Since each `yaml` simulations does not depend on the others, we can parallelize the process to speed things up:
```python
from multiprocessing import Pool

n_procs = 5 # number of cores available

with Pool(n_procs) as pool:
    pool.map(make_sim, paramlist)
```

In [ ]:
# https://techwiser.com/how-many-cores-does-my-cpu-have/
n_procs = 6 # number of cores available